In [3]:
#Figure for the manifold plot

#The idea is to save the final 4100 state trajectory from the algorithm, 
#and at each point simulate forward in time without control to ensure that 
#at every timestep (if control were to be lost), we would escape to the right. 
#Do this for both the Earth Moon system and the Saturn Enceladus system

In [4]:
using LinearAlgebra
using DifferentialEquations
using Plots
using DelimitedFiles

In [5]:
#parameters for Earth Moon System
μ = 1.215e-2
μ1 = 1-μ
μ2 = μ

#position of mass 1
pose_m1 = [-μ, 0, 0]

#position of mass 2
pose_m2 = [1-μ, 0, 0]

#value obtained from 3BP textbook (original)
L = 3.850e5 #in km - distance between centers of m1 and m2
#V_ = 1.025 #in km/s - orbital velocity of m1
#T_ = 2.361e6 #in seconds - orbital period of m1 and m2


#working
#########################################################################################
#updated for time unit to be in days
T_ = 2.361e6/86400 #in days - orbital period of m1 and m2
time_scale = T_/(2*pi)

#time unit of days
V_ = L/time_scale
acceleration_scale = L/((time_scale)^2)
##########################################################################################

#state of the L1 and L2 Lagrange points (computed offline)
XL1 = [0.8369180073169304, 0, 0, 0, 0, 0]
XL2 = [1.1556799130947355, 0, 0, 0, 0, 0]

6-element Vector{Float64}:
 1.1556799130947355
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
function effective_potential(X)
    x = X[1]
    y = X[2]
    z = X[3]

    r1 = sqrt((x+μ2)^2 + y^2 + z^2) 
    r2 = sqrt((x-μ1)^2 + y^2 + z^2)
    #assuming m3 is unit mass
   
    U = (-μ1/r1)-(μ2/r2)-0.5*(x^2+y^2)
   
    return U
    
end

effective_potential (generic function with 1 method)

In [7]:
function three_body_prob_dynamics(x)
    
    q = zeros(eltype(x),3)
    v = zeros(eltype(x),3)
    q = x[1:3]
    v = x[4:6]

    ẋ = zeros(eltype(x),6)
    U_q = zeros(eltype(x),3)
    
    U_q = (ForwardDiff.gradient(_x -> effective_potential(_x), q))
        
    ẋ[1:3] = v
    ẋ[4] = 2*v[2] - U_q[1] 
    ẋ[5] = -2*v[1] - U_q[2]
    ẋ[6] = -U_q[3]

    return ẋ
end

three_body_prob_dynamics (generic function with 1 method)

In [8]:
#inputs are in the custom SI unit scale

function three_body_prob_dynamics_scaled(x)

    q_original = zeros(eltype(x),3)
    v_original = zeros(eltype(x),3)
    
    q_original = x[1:3]/L 
    v_original = x[4:6]/V_
    
    x_original = [q_original; v_original]

    #original is in the CR3BP units
    ẋ_original = zeros(eltype(x),6)

    #calculate the original xdot (no scaling)

    ẋ_original = three_body_prob_dynamics(x_original)

    #then scale the output
    v_scaled = ẋ_original[1:3]*V_
    
    a_scaled = ẋ_original[4:6]*acceleration_scale

    ẋ_scaled = [v_scaled; a_scaled]

    return ẋ_scaled

end

three_body_prob_dynamics_scaled (generic function with 1 method)

In [9]:
#solve using DifferentialEquations.jl
function justdynamics_DFJL!(du, u, p, t)
    
    du[1:6] = three_body_prob_dynamics_scaled(u[1:6])
    
end

justdynamics_DFJL! (generic function with 1 method)

In [10]:
#gets the state and for the entire solution

function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

get_state (generic function with 1 method)

In [11]:
#integrate just the dynamics
function just_dynamics_integrate(x_0, period)
    
    tspan = (0.0, period)
    prob = ODEProblem(justdynamics_DFJL!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-12, reltol=1e-12)
    
    return sol
    
end

just_dynamics_integrate (generic function with 1 method)

In [12]:
#whole x trajectory
xtraj  = readdlm("xtraj_earthmoon.txt", '\t', Float64, '\n')'

ErrorException: at row 1, column 1 : ErrorException("file entry \"431250.331141646,431267.3850435738,431338.5415198259,431516.1039954271,431866.40665989975,432450.5125117846,433310.728696772,434464.6165560407,435904.6812598356,437601.2028120358,439506.32979740493,441558.41861573065,443686.20541498956,445812.717742762,447858.9683829417,449747.4661552481,451405.5716033627,452768.66428784316,453783.0326320827,454408.35537773086,454619.62344869104,454408.35815591115,453783.0347539939,452768.66458621924,451405.56898441474,449747.4596280711,447858.95708275365,445812.7009469847,443686.1825513256,441558.38825250673,439506.28685479064,437601.14204010024,435904.59833751054,434464.5080538236,433310.591911185,432450.34507837036,431866.2058196448,431515.8654027265,431338.2576154714,431267.04332981986,431249.96056335757,431249.9573381577,431267.0477856836,431338.2681138214,431515.88071867777,431866.2246762417,432450.36617339094,433310.6139809285,434464.52993705106,435904.61902324116,437601.160697860

In [ ]:
#at each of these points in x traj, simulate forward by a bit and hopefully all escape to the right ..